In [1]:
import base64
import datetime
import json
import os
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim

ModuleNotFoundError: No module named 'geopy'

We found student's assessment data for the state of california.  The data contains summary columns and detail columns.
It contains state summary data, County summary data and District summary data as well as detailed per school.
Since our analysis will be done at the District level, we created a script to read the assessment data and extract only the district summary data.

Which meant removing state and county data as well as school detailed data.
The file has 3 main columns that function as index.
County Code   Distric Code  School Code
00            00000         0000000       - This is the State Data
01 - 58       00000         0000000       - This is the County Summary Data
01-58         Various       0000000       - This is the District data which we are interested in
01-58         Various       Various       - This is the data per school

In [ ]:
file1 = open('commondata/sb_ca2017_all_csv_v2.txt', 'r') 
Lines = file1.readlines()
file2 = open('cleaneddata.txt', 'w') 
for x in range(0,3264975):
    #pdb.set_trace()
    #pdb.set_trace()
    if "County Code" in Lines[x].split(",")[0]:
        print(Lines[x])
        file2.writelines(Lines[x]) 
    elif '"00"' == Lines[x].split(",")[0]:
        continue
    elif '"00000"' == Lines[x].split(",")[1]:
        #then skip
        continue
    elif '"0000000"' != Lines[x].split(",")[2]:
        continue
        #then skip
        #file2.writelines(Lines[x])
        #print(Lines[x])
    #print(Lines[x])
    file2.writelines(Lines[x]) 

Afterwards we created a data frame with the new cleaned data.
The data we have for assessment has information on the Grades and also has a subgroup code which has ethnic and other groups information which we will need to separate.

In [2]:

# Read the data file and take a look at the data
df = pd.read_csv('cleaneddata.txt', sep=",")

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,1,2,3,4,5,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head(100)

,County Code,District Code,School Code,Filler,Test Year,Subgroup ID,Test Type,Total Tested At Entity Level,Total Tested with Scores,Grade,...,Area 1 Percentage Below Standard,Area 2 Percentage Above Standard,Area 2 Percentage Near Standard,Area 2 Percentage Below Standard,Area 3 Percentage Above Standard,Area 3 Percentage Near Standard,Area 3 Percentage Below Standard,Area 4 Percentage Above Standard,Area 4 Percentage Near Standard,Area 4 Percentage Below Standard
0,County Code,District Code,School Code,Filler,Test Year,Subgroup ID,Test Type,Total Tested At Entity Level,Total Tested with Scores,Grade,...,Area 1 Percentage Below Standard,Area 2 Percentage Above Standard,Area 2 Percentage Near Standard,Area 2 Percentage Below Standard,Area 3 Percentage Above Standard,Area 3 Percentage Near Standard,Area 3 Percentage Below Standard,Area 4 Percentage Above Standard,Area 4 Percentage Near Standard,Area 4 Percentage Below Standard
1,01,10017,0000000,NaN,2017,1,B,51,51,6,...,*,*,*,*,*,*,*,*,*,*
2,01,10017,0000000,NaN,2017,1,B,44,44,6,...,*,*,*,*,*,*,*,*,*,*
3,01,10017,0000000,NaN,2017,1,B,51,51,7,...,*,*,*,*,*,*,*,*,*,*
4,01,10017,0000000,NaN,2017,1,B,44,44,7,...,*,*,*,*,*,*,*,*,*,*
5,01,10017,0000000,NaN,2017,1,B,44,44,8,...,*,*,*,*,*,*,*,*,*,*
6,01,10017,0000000,NaN,2017,1,B,51,51,8,...,*,*,*,*,*,*,*,*,*,*
7,01,10017,0000000,NaN,2017,1,B,51,51,11,...,97.67,0.00,13.95,86.05,0.00,46.51,53.49,0.00,0.00,0.00
8,01,10017,0000000,NaN,2017,1,B,44,44,11,...,70.27,2.70,13.51,83.78,5.26,42.11,52.63,0.00,21.62,78.38
9,01,10017,0000000,NaN,2017,1,B,51,51,13,...,98.00,0.00,12.00,88.00,0.00,42.00,58.00,0.00,0.00,0.00


In [4]:
#We will drop the first row since it is duplicating the column names
df.drop([0], axis=0, inplace=True)

In [5]:
#We will drop the filler column, this column has no information, it is only blank spaces
df.drop(['Filler'], axis=1 , inplace = True)

In [6]:
#since we have district summary data, the school code of 0000000 is not useful, and we will remove that column
df.drop(['School Code'], axis=1, inplace = True)

In [7]:
df.columns

Index(['County Code', 'District Code', 'Test Year', 'Subgroup ID', 'Test Type',
       'Total Tested At Entity Level', 'Total Tested with Scores', 'Grade',
       'Test Id', 'CAASPP Reported Enrollment', 'Students Tested',
       'Mean Scale Score', 'Percentage Standard Exceeded',
       'Percentage Standard Met', 'Percentage Standard Met and Above',
       'Percentage Standard Nearly Met', 'Percentage Standard Not Met',
       'Students with Scores', 'Area 1 Percentage Above Standard',
       'Area 1 Percentage Near Standard', 'Area 1 Percentage Below Standard',
       'Area 2 Percentage Above Standard', 'Area 2 Percentage Near Standard',
       'Area 2 Percentage Below Standard', 'Area 3 Percentage Above Standard',
       'Area 3 Percentage Near Standard', 'Area 3 Percentage Below Standard',
       'Area 4 Percentage Above Standard', 'Area 4 Percentage Near Standard',
       'Area 4 Percentage Below Standard'],
      dtype='object')

In [8]:
df.drop(['Area 1 Percentage Above Standard',
       'Area 1 Percentage Near Standard', 'Area 1 Percentage Below Standard',
       'Area 2 Percentage Above Standard', 'Area 2 Percentage Near Standard',
       'Area 2 Percentage Below Standard', 'Area 3 Percentage Above Standard',
       'Area 3 Percentage Near Standard', 'Area 3 Percentage Below Standard',
       'Area 4 Percentage Above Standard', 'Area 4 Percentage Near Standard',
       'Area 4 Percentage Below Standard'], axis=1, inplace=True)

In [9]:
df.columns

Index(['County Code', 'District Code', 'Test Year', 'Subgroup ID', 'Test Type',
       'Total Tested At Entity Level', 'Total Tested with Scores', 'Grade',
       'Test Id', 'CAASPP Reported Enrollment', 'Students Tested',
       'Mean Scale Score', 'Percentage Standard Exceeded',
       'Percentage Standard Met', 'Percentage Standard Met and Above',
       'Percentage Standard Nearly Met', 'Percentage Standard Not Met',
       'Students with Scores'],
      dtype='object')

In [10]:
df.drop(['Test Year','Test Type'], axis=1, inplace=True)

In [11]:
#then we will reset the index and drop index column
df=df.reset_index()
df.head()

,index,County Code,District Code,Subgroup ID,Total Tested At Entity Level,Total Tested with Scores,Grade,Test Id,CAASPP Reported Enrollment,Students Tested,Mean Scale Score,Percentage Standard Exceeded,Percentage Standard Met,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,Students with Scores
0,1,01,10017,1,51,51,6,2,*,*,*,*,*,*,*,*,*
1,2,01,10017,1,44,44,6,1,*,*,*,*,*,*,*,*,*
2,3,01,10017,1,51,51,7,2,*,*,*,*,*,*,*,*,*
3,4,01,10017,1,44,44,7,1,*,*,*,*,*,*,*,*,*
4,5,01,10017,1,44,44,8,1,*,*,*,*,*,*,*,*,*


In [12]:
df.drop(['index'], axis = 1, inplace=True)
df.head()

,County Code,District Code,Subgroup ID,Total Tested At Entity Level,Total Tested with Scores,Grade,Test Id,CAASPP Reported Enrollment,Students Tested,Mean Scale Score,Percentage Standard Exceeded,Percentage Standard Met,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,Students with Scores
0,01,10017,1,51,51,6,2,*,*,*,*,*,*,*,*,*
1,01,10017,1,44,44,6,1,*,*,*,*,*,*,*,*,*
2,01,10017,1,51,51,7,2,*,*,*,*,*,*,*,*,*
3,01,10017,1,44,44,7,1,*,*,*,*,*,*,*,*,*
4,01,10017,1,44,44,8,1,*,*,*,*,*,*,*,*,*


In [13]:
df=df.apply(pd.to_numeric, errors='coerce')

In [102]:
df_stdtested=df[df['Subgroup ID']==1].groupby(['District Code'])['Students Tested'].sum()

In [15]:
df_stdtested

District Code
10017      177.0
10033      486.0
10041      750.0
10058     1024.0
10066        0.0
10074      530.0
10082      670.0
10090     1682.0
10108      706.0
10116      126.0
10124      194.0
10132      224.0
10140      228.0
10157     5562.0
10165      137.0
10173       36.0
10181        0.0
10199     2734.0
10207     1004.0
10215       21.0
10223        0.0
10231      100.0
10249      916.0
10256       26.0
10264        0.0
10272      924.0
10280      234.0
10298     2312.0
10306     4858.0
10314      656.0
          ...   
75697     1668.0
75713    33566.0
75739    28322.0
76414     3224.0
76455     1372.0
76497     1316.0
76505    54664.0
76513     1368.0
76547      624.0
76562     1356.0
76687     3850.0
76760    10140.0
76778     4786.0
76786    28460.0
76794     4476.0
76802     2686.0
76828    11686.0
76836     5676.0
76851     5764.0
76869     7056.0
76877     1542.0
76885      828.0
76893     1410.0
76901      776.0
76935      214.0
76943      762.0
76950      626.0


In [16]:
df_stdtested.to_csv (r'tot_students_tested.csv', index = True, header=True)

In [16]:
#when we run the df Grade unique code we saw that there were grades in text and others in numbers, so we replaced the values to be all numbers

#df['Grade'].replace(to_replace ="6", value =6, inplace=True) 
#df['Grade'].replace(to_replace ="7", value =7, inplace=True)
#df['Grade'].replace(to_replace ="8", value =8, inplace=True)
#df['Grade'].replace(to_replace ="11", value =11, inplace=True)
#df['Grade'].replace(to_replace ="4", value =4, inplace=True)
#df['Grade'].replace(to_replace ="5", value =5, inplace=True)
#df['Grade'].replace(to_replace ="3", value =3, inplace=True)
#df['Grade'].replace(to_replace ="13", value =13, inplace=True)



In [14]:
df['Grade'].unique()

array([ 6,  7,  8, 11, 13,  4,  5,  3])

In [15]:
df.astype({'Grade': 'int64'}).dtypes

County Code                            int64
District Code                          int64
Subgroup ID                            int64
Total Tested At Entity Level         float64
Total Tested with Scores             float64
Grade                                  int64
Test Id                                int64
CAASPP Reported Enrollment           float64
Students Tested                      float64
Mean Scale Score                     float64
Percentage Standard Exceeded         float64
Percentage Standard Met              float64
Percentage Standard Met and Above    float64
Percentage Standard Nearly Met       float64
Percentage Standard Not Met          float64
Students with Scores                 float64
dtype: object

In [16]:
df['Grade'].dtype

dtype('int64')

In [17]:
df['Subgroup ID'].dtype

dtype('int64')

In [18]:
df['Subgroup ID'].unique()

array([  1,   3,   4,   6,   7,   8,  31,  74,  76,  77,  78,  79,  80,
        90,  91,  92,  93,  94,  99, 111, 121, 128, 142, 144, 160, 170,
       180, 200, 202, 204, 205, 206, 207, 220, 222, 223, 224, 225, 226,
       227, 120, 203,  75, 201, 221, 190,  28])

In [19]:
df['County Code'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58])

In [20]:
df.head(29)

,County Code,District Code,Subgroup ID,Total Tested At Entity Level,Total Tested with Scores,Grade,Test Id,CAASPP Reported Enrollment,Students Tested,Mean Scale Score,Percentage Standard Exceeded,Percentage Standard Met,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,Students with Scores
0,1,10017,1,51.0,51.0,6,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,10017,1,44.0,44.0,6,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,10017,1,51.0,51.0,7,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,10017,1,44.0,44.0,7,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,10017,1,44.0,44.0,8,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,10017,1,51.0,51.0,8,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,10017,1,51.0,51.0,11,2,151.0,44.0,2412.8,0.0,0.00,0.00,4.55,95.45,44.0
7,1,10017,1,44.0,44.0,11,1,150.0,38.0,2435.4,0.0,5.26,5.26,21.05,73.68,38.0
8,1,10017,1,51.0,51.0,13,2,177.0,51.0,NaN,0.0,0.00,0.00,3.92,96.08,51.0
9,1,10017,1,44.0,44.0,13,1,175.0,44.0,NaN,0.0,4.55,4.55,18.18,77.27,44.0


In [21]:
df_subgroups= pd.read_csv('Subgroups.txt', sep=',')
df_subgroups = df_subgroups.set_index('id')

In [22]:
df_subgroups

,subgroup_id,subgroup_definition,subgroup_type
id,,,
1,1,"""All Students""","""All Students"""
3,3,"""Males""","""Gender"""
4,4,"""Females""","""Gender"""
6,6,"""Fluent-English Proficient and English Only""","""English-Language Fluency"""
7,7,"""Initially-Fluent English Proficient (IFEP)""","""English-Language Fluency"""
8,8,"""Reclassified-Fluent English Proficient (RFEP)""","""English-Language Fluency"""
28,28,"""Education""","""Migrant"""
31,31,"""EconomicallMigranty Disadvantaged""","""Economic Status"""
74,74,"""Black or African American""","""Ethnicity"""


In [26]:
df['Students Tested'].unique()

array([   nan,    44.,    38., ...,  3854.,  4131.,  4366.])

In [27]:
df.groupby(df['County Code']).size()

County Code
1     12541
2       290
3      1040
4      6578
5      2130
6      2144
7     10292
8      1082
9      6636
10    15555
11     3704
12    10790
13     6753
14     2580
15    21299
16     6308
17     3544
18     3466
19    50062
20     4736
21     7504
22      666
23     5451
24     9634
25     1504
26     1120
27    10696
28     2582
29     3960
30    16763
31     8127
32      814
33    16310
34     8754
35     3408
36    20085
37    22316
38     1208
39     9331
40     5836
41    11972
42     8946
43    17001
44     5076
45     9813
46      420
47     7219
48     4510
49    16979
50    13464
51     5116
52     5428
53     2470
54    18186
55     4098
56    10503
57     3590
58     2600
dtype: int64

In [30]:
ethnicity_sg= [74,75,76,77,78,79,80,144]
#df.loc[df['Language'].apply(lambda x: x in ['C', 'C++'] )]
assessment = df.loc[df['Subgroup ID'].apply(lambda x: x in ethnicity_sg)]

In [31]:
assessment.head(20)

,County Code,District Code,Subgroup ID,Total Tested At Entity Level,Total Tested with Scores,Grade,Test Id,CAASPP Reported Enrollment,Students Tested,Mean Scale Score,Percentage Standard Exceeded,Percentage Standard Met,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,Students with Scores
58,1,10017,74,23.0,23.0,6,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,1,10017,74,21.0,21.0,6,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,1,10017,74,21.0,21.0,7,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,1,10017,74,23.0,23.0,7,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
62,1,10017,74,21.0,21.0,8,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,1,10017,74,23.0,23.0,8,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,1,10017,74,23.0,23.0,11,2,72.0,22.0,2403.7,0.0,0.0,0.0,0.00,100.00,22.0
65,1,10017,74,21.0,21.0,11,1,72.0,20.0,2419.6,0.0,0.0,0.0,15.00,85.00,20.0
66,1,10017,74,23.0,23.0,13,2,85.0,23.0,NaN,0.0,0.0,0.0,0.00,100.00,23.0
67,1,10017,74,21.0,21.0,13,1,85.0,21.0,NaN,0.0,0.0,0.0,14.29,85.71,21.0


In [32]:
assessment.columns

Index(['County Code', 'District Code', 'Subgroup ID',
       'Total Tested At Entity Level', 'Total Tested with Scores', 'Grade',
       'Test Id', 'CAASPP Reported Enrollment', 'Students Tested',
       'Mean Scale Score', 'Percentage Standard Exceeded',
       'Percentage Standard Met', 'Percentage Standard Met and Above',
       'Percentage Standard Nearly Met', 'Percentage Standard Not Met',
       'Students with Scores'],
      dtype='object')

In [35]:
#Converting all the assessment columns from percentage columns to absolute numbers for exceeded, met,met & above, nearly met, not met
assessment['Total Standard Exceeded'] =(assessment['Students Tested']*assessment['Percentage Standard Exceeded']/100).round(decimals=0)
assessment['Total Standard Met'] =(assessment['Students Tested']*assessment['Percentage Standard Met']/100).round(decimals=0)
assessment['Total Standard Met and Above'] =(assessment['Students Tested']*assessment['Percentage Standard Met and Above']/100).round(decimals=0)
assessment['Total Standard Nearly Met'] =(assessment['Students Tested']*assessment['Percentage Standard Nearly Met']/100).round(decimals=0)
assessment['Total Standard Not Met'] =(assessment['Students Tested']*assessment['Percentage Standard Not Met']/100).round(decimals=0)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

In [27]:
assessment.columns

Index(['County Code', 'District Code', 'Subgroup ID',
       'Total Tested At Entity Level', 'Total Tested with Scores', 'Grade',
       'Test Id', 'CAASPP Reported Enrollment', 'Students Tested',
       'Mean Scale Score', 'Percentage Standard Exceeded',
       'Percentage Standard Met', 'Percentage Standard Met and Above',
       'Percentage Standard Nearly Met', 'Percentage Standard Not Met',
       'Students with Scores', 'Total Standard Exceeded', 'Total Standard Met',
       'Total Standard Met and Above', 'Total Standard Nearly Met',
       'Total Standard Not Met'],
      dtype='object')

In [33]:
df_mean=assessment.groupby(['County Code','District Code','Subgroup ID']).agg({'Mean Scale Score':'mean'}).reset_index()

In [28]:
#assessment = assessment.drop(columns=['Percentage Standard Exceeded','Percentage Standard Met and Above', 'Percentage Standard Met','Percentage Standard Met and Above','Percentage Standard Nearly Met','Percentage Standard Not Met'])

In [29]:
#This is a file created to look at Mean Scale Score within all grades and subgroups before grouping by county and district code
#assessment=assessment.dropna()
#assessment.to_csv('assessment_all_grades_test.csv')

In [37]:
df_ethnicity_new=assessment.groupby(['County Code','District Code','Subgroup ID'])['Total Tested At Entity Level', 'Total Tested with Scores', 'CAASPP Reported Enrollment', 'Students Tested',
       'Total Standard Exceeded',
       'Total Standard Met', 'Total Standard Met and Above',
       'Total Standard Nearly Met', 'Total Standard Not Met',
       'Students with Scores'].sum()

In [38]:
df_ethnicity_new.head()

Total Tested At Entity Level  \
County Code District Code Subgroup ID                                 
1           10017         74                                  220.0   
                          76                                    0.0   
                          77                                    0.0   
                          78                                  175.0   
                          79                                    0.0   

                                       Total Tested with Scores  \
County Code District Code Subgroup ID                             
1           10017         74                              220.0   
                          76                                0.0   
                          77                                0.0   
                          78                              175.0   
                          79                                0.0   

                                       CAASPP Reported Enrollment  \
County Code District Code Subgroup ID                               
1           10017         74                                314.0   
                          76                                  0.0   
                          77                                  0.0   
                          78                                219.0   
                          79                                  0.0   

                                       Students Tested  \
County Code District Code Subgroup ID                    
1           10017         74                      86.0   
                          76                       0.0   
                          77                       0.0   
                          78                      63.0   
                          79                       0.0   

                                       Total Standard Exceeded  \
County Code District Code Subgroup ID                            
1           10017         74                               0.0   
                          76                               0.0   
                          77                               0.0   
                          78                               0.0   
                          79                               0.0   

                                       Total Standard Met  \
County Code District Code Subgroup ID                       
1           10017         74                          0.0   
                          76                          0.0   
                          77                          0.0   
                          78                          2.0   
                          79                          0.0   

                                       Total Standard Met and Above  \
County Code District Code Subgroup ID                                 
1           10017         74                                    0.0   
                          76                                    0.0   
                          77                                    0.0   
                          78                                    2.0   
                          79                                    0.0   

                                       Total Standard Nearly Met  \
County Code District Code Subgroup ID                              
1           10017         74                                 6.0   
                          76                                 0.0   
                          77                                 0.0   
                          78                                10.0   
                          79                                 0.0   

                                       Total Standard Not Met  \
County Code District Code Subgroup ID                           
1           10017         74                             80.0   
                          76                              0.0   
                          77                          

In [40]:
df_ethnicity_new=df_ethnicity_new.reset_index()

In [41]:
new_df=pd.merge(df_mean, df_ethnicity_new, how='outer', on=['County Code','District Code','Subgroup ID'])
new_df= new_df.sort_values(by=['District Code'])

In [43]:
new_df.dropna()

,County Code,District Code,Subgroup ID,Mean Scale Score,Total Tested At Entity Level,Total Tested with Scores,CAASPP Reported Enrollment,Students Tested,Total Standard Exceeded,Total Standard Met,Total Standard Met and Above,Total Standard Nearly Met,Total Standard Not Met,Students with Scores
0,1,10017,74,2411.650000,220.0,220.0,314.0,86.0,0.0,0.0,0.0,6.0,80.0,86.0
3,1,10017,78,2448.100000,175.0,175.0,219.0,63.0,0.0,2.0,2.0,10.0,51.0,63.0
161,3,10033,80,2480.750000,1368.0,1368.0,312.0,279.0,4.0,37.0,41.0,61.0,177.0,279.0
175,4,10041,78,2523.900000,816.0,816.0,148.0,136.0,2.0,18.0,20.0,39.0,77.0,136.0
176,4,10041,80,2492.983333,1624.0,1624.0,354.0,338.0,20.0,41.0,61.0,105.0,172.0,338.0
273,5,10058,80,2484.235714,2976.0,2976.0,804.0,744.0,84.0,148.0,232.0,232.0,280.0,744.0
334,7,10074,78,2401.150000,623.0,623.0,182.0,163.0,0.0,3.0,3.0,18.0,142.0,163.0
331,7,10074,74,2370.500000,768.0,768.0,186.0,163.0,0.0,3.0,3.0,9.0,151.0,163.0
479,8,10082,78,2474.150000,432.0,432.0,106.0,76.0,1.0,3.0,4.0,19.0,53.0,76.0
480,8,10082,80,2491.100000,1936.0,1928.0,544.0,445.0,12.0,86.0,98.0,117.0,228.0,443.0


In [68]:
#def calculate_label(ethnicity):
#    if ethnicity ==74:
#        return 'black_african_american'
#   elif ethnicity == 75:
#       return 'american indian_Alaska native'
#   elif ethnicity == 76:
#       return 'asian'
#   elif ethnicity == 77:
#       return 'filipino'
#   elif ethnicity == 78:
#       return 'hispanic_latin'
#   elif ethnicity == 79:
#       return 'native hawaian_pacific islander'
#   elif ethnicity == 80:        return 'white'
#    elif ethnicity == 144:
#        return 'two or more races'
#    else:
#        return 'nan'
    
#df_ethnicity_new['ethnicity']= df_ethnicity_new['Subgroup ID'].apply(calculate_label)
#df_ethnicity_new.columns

In [62]:
#create dummy variables for subgroup
#new_df['Subgroup ID']=new_df['Subgroup ID'].astype('category')

In [64]:
#new_df=pd.get_dummies(new_df,dummy_na=True)

In [44]:
new_df=new_df.dropna()

In [45]:
new_df.head(10)

,County Code,District Code,Subgroup ID,Mean Scale Score,Total Tested At Entity Level,Total Tested with Scores,CAASPP Reported Enrollment,Students Tested,Total Standard Exceeded,Total Standard Met,Total Standard Met and Above,Total Standard Nearly Met,Total Standard Not Met,Students with Scores
0,1,10017,74,2411.650000,220.0,220.0,314.0,86.0,0.0,0.0,0.0,6.0,80.0,86.0
3,1,10017,78,2448.100000,175.0,175.0,219.0,63.0,0.0,2.0,2.0,10.0,51.0,63.0
161,3,10033,80,2480.750000,1368.0,1368.0,312.0,279.0,4.0,37.0,41.0,61.0,177.0,279.0
175,4,10041,78,2523.900000,816.0,816.0,148.0,136.0,2.0,18.0,20.0,39.0,77.0,136.0
176,4,10041,80,2492.983333,1624.0,1624.0,354.0,338.0,20.0,41.0,61.0,105.0,172.0,338.0
273,5,10058,80,2484.235714,2976.0,2976.0,804.0,744.0,84.0,148.0,232.0,232.0,280.0,744.0
334,7,10074,78,2401.150000,623.0,623.0,182.0,163.0,0.0,3.0,3.0,18.0,142.0,163.0
331,7,10074,74,2370.500000,768.0,768.0,186.0,163.0,0.0,3.0,3.0,9.0,151.0,163.0
479,8,10082,78,2474.150000,432.0,432.0,106.0,76.0,1.0,3.0,4.0,19.0,53.0,76.0
480,8,10082,80,2491.100000,1936.0,1928.0,544.0,445.0,12.0,86.0,98.0,117.0,228.0,443.0


In [46]:
new_df.to_csv (r'assessmentbyethnicity.csv', index = False, header=True)

In [52]:
#df.head(40)

,County Code,District Code,Subgroup ID,Grade,Test Id,CAASPP Reported Enrollment,Students Tested,Mean Scale Score,Percentage Standard Exceeded,Percentage Standard Met,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,Students with Scores
0,1,10017,1,6,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,10017,1,6,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,10017,1,7,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,10017,1,7,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,10017,1,8,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,10017,1,8,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,10017,1,11,2,151.0,44.0,2412.8,0.0,0.00,0.00,4.55,95.45,44.0
7,1,10017,1,11,1,150.0,38.0,2435.4,0.0,5.26,5.26,21.05,73.68,38.0
8,1,10017,1,13,2,177.0,51.0,NaN,0.0,0.00,0.00,3.92,96.08,51.0
9,1,10017,1,13,1,175.0,44.0,NaN,0.0,4.55,4.55,18.18,77.27,44.0


In [37]:
#df_math_assessment = df_ethnicity[df_ethnicity['Test Id']==2]
#df_litheracy_assessment = df_ethnicity[df_ethnicity['Test Id']==1]

In [39]:
#df_math_assessment['Subgroup ID'].unique()

array([  1,   3,   4,   6,   7,   8,  31,  74,  76,  77,  78,  79,  80,
        90,  91,  92,  93,  94,  99, 111, 121, 128, 142, 144, 160, 170,
       180, 200, 202, 204, 205, 206, 207, 220, 222, 223, 224, 225, 226,
       227, 120, 203,  75, 201, 221, 190,  28])

In [38]:
#df_math_assessment.head()

,County Code,District Code,Subgroup ID,Grade,Test Id,CAASPP Reported Enrollment,Students Tested,Mean Scale Score,Percentage Standard Exceeded,Percentage Standard Met,Percentage Standard Met and Above,Percentage Standard Nearly Met,Percentage Standard Not Met,Students with Scores
0,1,10017,1,6,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,10017,1,7,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,10017,1,8,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,10017,1,11,2,151.0,44.0,2412.8,0.0,0.0,0.0,4.55,95.45,44.0
8,1,10017,1,13,2,177.0,51.0,NaN,0.0,0.0,0.0,3.92,96.08,51.0


In [186]:
#df_litheracy_assessment['County Code'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58])

In [120]:
#df_math_g3.to_csv (r'math_g3_district.csv', index = False, header=True)